# Module 1 Load Data

In [1]:
from collections import defaultdict
import numpy as np
import pandas as pd

In [30]:
## load dataset & filter out missing data
# dbfile = pd.read_csv('./Data/vdjdb.slim.txt', sep='\t')
dbfile = pd.read_csv('./Data/VDJDB/vdjdb_full.txt', sep='\t')
# print(dbfile.columns)
dbfile = dbfile[['cdr3.alpha', 'cdr3.beta','antigen.epitope','antigen.species',"mhc.class"]]
print(dbfile.shape)
dbfile = dbfile.dropna(subset=['cdr3.alpha', 'cdr3.beta','antigen.epitope'])
print(dbfile.shape)
dbfile.head()

(60055, 5)
(29266, 5)


C:\Users\croco\AppData\Local\Temp\ipykernel_4168\3075150544.py:3: DtypeWarning: Columns (20,29,30) have mixed types. Specify dtype option on import or set low_memory=False.
  dbfile = pd.read_csv('./Data/VDJDB/vdjdb_full.txt', sep='\t')


,cdr3.alpha,cdr3.beta,antigen.epitope,antigen.species,mhc.class
0,CIVRAPGRADMRF,CASSYLPGQGDHYSNQPQHF,FLKEKGGL,HIV-1,MHCI
2,CAVPSGAGSYQLTF,CASSFEPGQGFYSNQPQHF,FLKEKGGL,HIV-1,MHCI
3,CAVKASGSRLT,CASSYEPGQVSHYSNQPQHF,FLKEKGGL,HIV-1,MHCI
4,CAYRPPGTYKYIF,CASSALASLNEQFF,FLKEKGGL,HIV-1,MHCI
5,CIVRAPGRADMRF,CASSYLPGQGDHYSNQPQHF,FLKEQGGL,HIV-1,MHCI


In [42]:
iedbfile = pd.read_csv('./Data/IEDB/tcell_receptor_table_export_1668026760.csv')
# print(iedbfile.columns)
# Chain 1 Full Sequence', 'Chain 1 Accession', 'Chain 1 CDR3 Curated',
#        'Chain 1 CDR3 Calculated'
iedbfile = iedbfile[['Chain 1 CDR3 Curated','Chain 2 CDR3 Curated',"Description","Organism"]]
print(iedbfile.shape)
iedbfile = iedbfile.dropna(subset=['Chain 1 CDR3 Curated','Chain 2 CDR3 Curated',"Description"])
print(iedbfile.shape)
iedbfile.head()

C:\Users\croco\AppData\Local\Temp\ipykernel_4168\919688929.py:1: DtypeWarning: Columns (8,10,11,13,15,16,17,18,19,20,21,22,23,24,25,30,31,36,37,42,44,45,46,47,48,49,50,51,52,53,54,59,60,65,66) have mixed types. Specify dtype option on import or set low_memory=False.
  iedbfile = pd.read_csv('./Data/IEDB/tcell_receptor_table_export_1668026760.csv')


(205824, 4)
(26231, 4)


,Chain 1 CDR3 Curated,Chain 2 CDR3 Curated,Description,Organism
0,IVVRSSNTGKLI,ASSQDRDTQY,VMAPRTLIL,Homo sapiens (human)
80,CAVTTDSWGKLQF,CASRPGLAGGRPEQYF,LLFGYPVYV,Human T-cell leukemia virus type I (Human T ce...
82,CAVTTDSWGKLQF,CASRPGLMSAQPEQYF,LLFGYPVYV,Human T-cell leukemia virus type I (Human T ce...
109,CAVRPTSGGSYIPTF,CASSYVGNTGELFF,SLLMWITQC,Homo sapiens (human)
202,TVYGGATNKLI,SARGGSYNSPLH,LSRFSWGAEGQRPGFGYGG,Homo sapiens (human)


In [12]:
from functions import load
# from functions.load import enc_list_bl_max_len, positional_coding, blosum50_20aa 

In [28]:
from importlib import reload
reload(load)

<module 'functions.load' from 'c:\\Users\\croco\\OneDrive - Emory University\\Courses\\BMI 536 Deep Learning\\FinalProj\\functions\\load.py'>

In [5]:
print(dbfile.head(n=2)["cdr3.alpha"])
load.enc_list_bl_max_len(dbfile.head(n=2)["cdr3.alpha"],load.blosum50_20aa,30).shape

0     CIVRAPGRADMRF
2    CAVPSGAGSYQLTF
Name: cdr3.alpha, dtype: object


(2, 30, 20)

In [10]:
tokens = load.blosum50_20aa.keys()

tokens = list(tokens)
tk_dict = defaultdict(lambda: None)
for idx in range(len(tokens)):
    tk_dict[tokens[idx]]=idx+1

In [43]:
load.positional_coding(dbfile.head(n=52)["cdr3.alpha"],tk_dict,30).shape
load.positional_coding(iedbfile.head(n=52)["Chain 1 CDR3 Curated"],tk_dict,30).shape

(52, 30)

In [46]:
max(iedbfile["Description"].str.len())

122

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train, test = train_test_split(dbfile, test_size=0.2)